In [1]:
#　listファイルをrawCSVファイルに変換したものをmodifiedCSVファイルに変換するもの

In [2]:
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog

In [3]:
def clean_csv_files(input_dir, output_dir=None, na_values_to_remove=['?-?']):
    """
    指定されたディレクトリ内のCSVファイルを読み込み、指定された値を欠損値として認識し、
    欠損値を含む行を削除して、CSV形式で保存します。
    処理の過程（欠損値の数など）も表示します。

    Args:
        input_dir (str): CSV形式のファイルが格納されているディレクトリのパス。
        output_dir (str, optional): クリーンアップされたCSVファイルを保存するディレクトリのパス。
                                    Noneの場合、入力ディレクトリと同じ場所に保存されます。
        na_values_to_remove (list): 欠損値として認識し、削除する文字列のリスト。
    """
    if not os.path.exists(input_dir):
        print(f"エラー: 指定された入力ディレクトリ '{input_dir}' が存在しません。")
        return

    if output_dir and not os.path.exists(output_dir):
        print(f"出力ディレクトリ '{output_dir}' が存在しないため、作成します。")
        os.makedirs(output_dir)
    elif output_dir is None:
        output_dir = input_dir

    print(f"\n--- CSVファイルクリーンアップ処理開始 ---")
    print(f"入力ディレクトリ: {input_dir}")
    print(f"出力ディレクトリ: {output_dir}")
    print(f"削除対象の文字列: {na_values_to_remove}")

    processed_files_count = 0
    # ディレクトリ内のすべてのCSVファイルを検索
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            input_filepath = os.path.join(input_dir, filename)
            
            print(f"\nファイルを処理中: {input_filepath}")

            try:
                # CSVを読み込む際に、指定された値を欠損値 (NaN) として認識させる
                # ヘッダーは通常あると仮定（read_csvのデフォルト）
                # '?-?' は 'Assignment' 列だけでなく、他の列にも現れる可能性があるので、
                # na_values にリストで渡すことで、データフレーム全体で適用されます。
                df = pd.read_csv(input_filepath, encoding='utf-8', na_values=na_values_to_remove)

                print(f"--- '{filename}' 欠損値として認識後のデータフレームの最初の数行 ---")
                print(df.head())
                
                # 欠損値（NaN）の数を確認する (削除前)
                missing_values_count = df.isnull().sum()
                # 欠損値が0より大きい列のみ表示
                print(f"\n--- '{filename}' 欠損値の数（削除前） ---")
                print(missing_values_count[missing_values_count > 0]) 

                initial_rows = len(df)
                print(f"元の行数: {initial_rows}")

                # 欠損値を含む行を削除する
                # how='any' は、いずれかの列にNaNがあれば行を削除
                df_cleaned = df.dropna(how='any')

                print(f"\n--- '{filename}' 欠損値を含む行削除後のデータフレームの最初の数行 ---")
                print(df_cleaned.head())

                cleaned_rows = len(df_cleaned)
                print(f"削除後の行数: {cleaned_rows}")
                print(f"削除された行数: {initial_rows - cleaned_rows}")

                # 出力ファイルパスの生成
                # 元のファイル名から拡張子と'_raw'の部分を除き、'_modified.csv' を追加

                # 1. 拡張子を除いたファイル名を取得 (例: 'hsqc_GDPList_raw')
                base_filename = os.path.splitext(filename)[0]

                # 2. もしファイル名が '_raw' で終わる場合、'_raw' を削除 (例: 'hsqc_GDPList')
                if base_filename.endswith('_raw'):
                    base_filename = base_filename[:-4] # '_raw' は4文字なので削除

                # 3. 新しい接尾辞 '_modified.csv' を追加
                output_filename = base_filename + "_modified.csv"
                # 例: 'hsqc_GDPList_raw.csv' -> 'hsqc_GDPList_modified.csv'
                # 例: 'another_file.csv' -> 'another_file_modified.csv' (元々_rawがない場合も対応)

                output_filepath = os.path.join(output_dir, output_filename)
                
                # データフレームをCSVファイルに保存
                df_cleaned.to_csv(output_filepath, index=False)
                print(f"'{filename}' が '{output_filepath}' にクリーンアップされて保存されました。")
                processed_files_count += 1

            except FileNotFoundError:
                print(f"エラー: ファイル '{input_filepath}' が見つかりません。")
            except Exception as e:
                print(f"ファイルの読み込みまたは処理中にエラーが発生しました '{filename}': {e}")
        else:
            print(f"スキップ: '{filename}' (拡張子が .csv ではありません)")
    print(f"\n--- CSVファイルクリーンアップ処理終了。処理されたファイル数: {processed_files_count} ---")

In [4]:
# --- 使用例 (フォルダ選択ダイアログ付き) ---
if __name__ == "__main__":
    # Tkinterのルートウィンドウを作成（非表示）
    root = tk.Tk()
    root.withdraw()

    print("--- CSVファイルクリーンアップ処理開始 ---")
    # クリーンアップ対象のCSVファイルがあるフォルダを選択
    csv_input_folder = filedialog.askdirectory(
        title="クリーンアップ対象のCSVファイルが格納されているフォルダを選択してください"
    )

    if not csv_input_folder:
        print("入力フォルダが選択されませんでした。処理を終了します。")
    else:
        # クリーンアップされたCSVファイルを保存する最終的なフォルダを選択
        cleaned_csv_output_folder = filedialog.askdirectory(
            title="クリーンアップされたCSVファイルを保存するフォルダを選択してください"
        )
        if not cleaned_csv_output_folder:
            print("出力フォルダが選択されませんでした。入力フォルダと同じ場所に保存します。")
            cleaned_csv_output_folder = csv_input_folder
        
        # clean_csv_files 関数を呼び出す
        clean_csv_files(csv_input_folder, cleaned_csv_output_folder, na_values_to_remove=['?-?'])


--- CSVファイルクリーンアップ処理開始 ---

--- CSVファイルクリーンアップ処理開始 ---
入力ディレクトリ: C:/JupyterNotebook/NMR_Folder/CSV_OriginalFolder/rawCSV/100
出力ディレクトリ: C:/JupyterNotebook/NMR_Folder/CSV_OriginalFolder/modifiedCSV/100mod
削除対象の文字列: ['?-?']

ファイルを処理中: C:/JupyterNotebook/NMR_Folder/CSV_OriginalFolder/rawCSV/100\GDP_noHis_100uM_raw.csv
--- 'GDP_noHis_100uM_raw.csv' 欠損値として認識後のデータフレームの最初の数行 ---
  Assignment       w1     w2  Data Height
0        NaN  113.201  6.900       151670
1        NaN  112.653  7.649       485996
2        NaN  112.648  6.918       531332
3        NaN  112.104  7.330       395443
4        NaN  111.785  7.411       453775

--- 'GDP_noHis_100uM_raw.csv' 欠損値の数（削除前） ---
Assignment    21
dtype: int64
元の行数: 183

--- 'GDP_noHis_100uM_raw.csv' 欠損値を含む行削除後のデータフレームの最初の数行 ---
   Assignment       w1     w2  Data Height
21      T2N-H  122.240  8.517       377924
22      E3N-H  126.853  8.304       352773
23      Y4N-H  121.801  8.757       257450
24      K5N-H  124.585  9.137       295339
25      L6N-H